# <center> PHẦN 2. KHÁM PHÁ DỮ LIỆU & ĐẶT CÂU HỎI

# Tổng quan về tiền xử lý dữ liệu
Mục tiêu chính:
## A. Làm sạch dữ liệu (Exploratory Data Analysis - EDA):
1. Khám phá dữ liệu: Đưa ra những đánh giá đầu tiên về dữ liệu để hiểu cấu trúc và ý nghĩa của chúng.
Trước khi xử lý, chúng ta cần hiểu rõ cấu trúc và nội dung của dữ liệu. Các hoạt động bao gồm:

+ Kiểm tra số lượng dòng, cột.
Xác định kiểu dữ liệu của từng cột (số, phân loại, ngày tháng, ...).
+ Kiểm tra tỷ lệ dữ liệu bị thiếu, các giá trị bất thường.
+ Tóm tắt các thống kê cơ bản như trung bình, giá trị nhỏ nhất/lớn nhất, phương sai, ...

(Bước này làm giống format HW2)

2. Xử lý dữ liệu thiếu  (Handling Missing Data):
Dữ liệu thiếu thường gây sai lệch trong phân tích. Chúng ta sẽ điền giá trị bị thiếu bằng các phương pháp như:
+ Giá trị trung bình/median.
+ Thuật toán Machine Learning. Ae chú ý là cần áp dụng đúng model, ví dụ như đối với kiểu dữ liệu là Category (phân lớp) thì nên sử dụng KNN, hoặc int64 (Liên tục) thì xài Linear Regression,....
+ Hoặc loại bỏ các dòng/cột chứa quá nhiều giá trị bị thiếu nếu cần thiết.

3. Xử lý dữ liệu nhiễu: Loại bỏ các giá trị bất thường (outliers) để đảm bảo dữ liệu sạch (Sử dụng  phương pháp như IQR (Interquartile Range) để phát hiện và loại bỏ những giá trị nằm ngoài phạm vi hợp lý.)

4. Chuẩn hóa kiểu dữ liệu:
- Chuyển đổi dữ liệu về đúng kiểu cần thiết (ví dụ: chuyển đổi ngày tháng, số nguyên, số thực).
- Mã hóa dữ liệu phân loại (categorical data) thành dạng số, phù hợp với mô hình (Label Encoding hoặc One-hot Encoding). (nếu cảm thấy cần thiết)
P/s: Thứ tự các bước trên có thể thay đổi, tùy theo ae thiết kế


## B. Giảm chiều dữ liệu(Dimensionality Reduction): Giảm độ phức tạp
1..Khám phá dữ liệu (Tiếp theo):
- Trực quan hóa mối quan hệ giữa các biến để xác định những biến dư thừa, không cần thiết.
- Sử dụng ma trận tương quan (Correlation Matrix) để xem xét độ tương quan giữa các biến:
  - Loại bỏ các biến có độ tương quan cao (vì có thể gây trùng lặp thông tin).
  - Giữ lại các biến quan trọng nhất để giảm độ phức tạp và tăng hiệu quả phân tích.

## C. Chuẩn hóa dữ liệu (Nếu còn thời gian)
- Min-Max Scaling: Đưa dữ liệu về khoảng [0, 1].
- Z-score Standardization: Chuẩn hóa dữ liệu thành phân phối chuẩn với trung bình bằng 0 và phương sai bằng 1.
- Robust Scaling: Loại bỏ tác động của các giá trị ngoại lai khi chuẩn hóa.
- Decimal Scaling: Chia giá trị cho lũy thừa của 10 để thu hẹp giá trị dữ liệu.

## D. Rời rạc hóa dữ liệu (Tìm hiểu thêm)

## E. Lưu dữ liệu sạch: Dữ liệu sau khi tiền xử lý sẽ được lưu lại để sử dụng trong các giai đoạn phân tích tiếp theo.

# Import các thư viện cần thiết


In [8]:
import requests
import json
import pandas as pd
import numpy as np
import time
import datetime
import calendar
import seaborn as sns
import matplotlib.pyplot as plt

# A.1. Khám phá dữ liệu

Đọc file dữ liệu ban đầu chưa xử lí từ raw_data

In [9]:
raw_match_statistics = pd.read_csv('./data/raw_data/match_statistics.csv', delimiter='\t')
raw_match_statistics.head()

,match_id,home_total_shots,home_shots_off_target,home_shots_saved,home_corners,home_free_kicks,home_offsides,home_fouls,away_total_shots,away_shots_off_target,away_shots_saved,away_corners,away_free_kicks,away_offsides,away_fouls,home_yellow_cards,home_red_cards,away_yellow_cards,away_red_cards
0,3050176,4.0,5.0,3.0,3.0,10.0,7.0,2.0,5.0,1.0,2.0,6.0,9.0,7.0,3.0,0.0,0.0,1.0,1.0
1,3050173,2.0,8.0,4.0,3.0,12.0,11.0,1.0,6.0,6.0,1.0,5.0,12.0,12.0,0.0,2.0,0.0,2.0,0.0
2,3050175,5.0,7.0,1.0,8.0,19.0,13.0,0.0,1.0,6.0,2.0,2.0,13.0,16.0,3.0,2.0,0.0,2.0,0.0
3,3050171,8.0,4.0,2.0,5.0,12.0,14.0,5.0,2.0,2.0,4.0,4.0,19.0,9.0,3.0,1.0,0.0,2.0,0.0
4,3050170,2.0,4.0,1.0,2.0,9.0,8.0,2.0,4.0,2.0,2.0,5.0,10.0,8.0,1.0,2.0,0.0,1.0,0.0


Dữ liệu có bao nhiêu hàng, bao nhiêu cột?



In [10]:
num_rows, num_cols = raw_match_statistics.shape
print(f'Số dòng: {num_rows}')
print(f'Số cột: {num_cols}')

Số dòng: 3800
Số cột: 19


Dữ liệu các dòng có bị lặp không?

In [11]:
duplicates = raw_match_statistics.duplicated()
print(f"Số dòng bị lặp: {duplicates.sum()}")

Số dòng bị lặp: 0


Ý nghĩa của các cột dữ liệu

Tỷ lệ thiếu dữ liệu

Tóm tắt các thống kê cơ bản như trung bình, giá trị nhỏ nhất/lớn nhất, phương sai

# A.2.Xử lý dữ liệu thiếu
Cái này ae tự tạo cell trình bày giống trong HW2, hoặc có thể thiết kế theo ý muốn, miễn sao tối đa hóa việc xử lý sao cho chuẩn chỉnh


# A.3. Xử lý dữ liệu nhiễu

Sử dụng phương pháp IQR, đơn giản là phát hiện điểm dữ liệu nằm bên ngoài khoảng tứ phân vị thì loại **bỏ**

Có gì ae tự tạo cell thêm tương ứng với cái section đã ghi trên mục đầu, những cái đã trình bày trên chỉ là format chung, mỗi người có thể thay đổi cell và trình bày tùy ý, mục đích chung là tối đa hóa việc làm sạch dữ liệu